# Load Processed Data into Vector Database

This notebook loads output from data prep kit into Milvus

**Step-4 in this workflow**

![](../media/rag-overview-2.png)


## Configuration

In [1]:

from my_config import MY_CONFIG

## Step-: Load Parquet Data

Load all  `.parquet` files in the given dir

In [2]:
import pandas as pd
import glob

print ('Loading data from : ', MY_CONFIG.OUTPUT_FOLDER_FINAL)

# Get a list of all Parquet files in the directory
parquet_files = glob.glob(f'{MY_CONFIG.OUTPUT_FOLDER_FINAL}/*.parquet')
print ("Number of parquet files to read : ", len(parquet_files))
print ()

# Create an empty list to store the DataFrames
dfs = []

# Loop through each Parquet file and read it into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    print (f"Read file: '{file}'.  number of rows = {df.shape[0]}")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data_df = pd.concat(dfs, ignore_index=True)

print (f"\nTotal number of rows = {data_df.shape[0]}")

Loading data from :  output-papers/output_final
Number of parquet files to read :  2

Read file: 'output-papers/output_final/attention-is-all-you-need.parquet'.  number of rows = 88
Read file: 'output-papers/output_final/Granite_Foundation_Models.parquet'.  number of rows = 178

Total number of rows = 266


In [3]:

## Shape the data

MY_CONFIG.EMBEDDING_LENGTH =  len(data_df.iloc[0]['embeddings'])
print ('embedding length: ', MY_CONFIG.EMBEDDING_LENGTH)

# rename 'embeddings' columns as 'vector' to match default schema
# if 'vector' not in data_df.columns and 'embeddings' in data_df.columns:
#     data_df = data_df.rename( columns= {'embeddings' : 'vector'})
# if 'text' not in data_df.columns and 'contents' in data_df.columns:
#     data_df = data_df.rename( columns= {'contents' : 'text'})

data_df = data_df.rename( columns= {'embeddings' : 'vector', 'contents' : 'text'})

print (data_df.info())
data_df.head(3)

embedding length:  384
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   filename          266 non-null    object 
 1   num_pages         266 non-null    int64  
 2   num_tables        266 non-null    int64  
 3   num_doc_elements  266 non-null    int64  
 4   document_id       266 non-null    object 
 5   ext               266 non-null    object 
 6   hash              266 non-null    object 
 7   size              266 non-null    int64  
 8   date_acquired     266 non-null    object 
 9   pdf_convert_time  266 non-null    float64
 10  source_filename   266 non-null    object 
 11  text              266 non-null    object 
 12  doc_jsonpath      266 non-null    object 
 13  page_number       266 non-null    int64  
 14  bbox              266 non-null    object 
 15  chunk_id          266 non-null    int64  
 16  removed           266

,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,text,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash,vector
0,attention-is-all-you-need.pdf,15,4,193,82b50c62-ad6d-4c6e-94f5-128d5c8141dc,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:36:45.815652,53.907655,attention-is-all-you-need.pdf,Attention Is All You Need\nAshish Vaswani ∗ Go...,$.main-text[3],1,"[116.55065918, 524.45581055, 216.03900146, 558...",0,[],-1,"[-0.026405312, -0.038113207, 0.0457838, 0.0024..."
1,attention-is-all-you-need.pdf,15,4,193,82b50c62-ad6d-4c6e-94f5-128d5c8141dc,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:36:45.815652,53.907655,attention-is-all-you-need.pdf,Attention Is All You Need\nNoam Shazeer ∗ Goog...,$.main-text[4],1,"[230.56285095, 524.74963379, 309.13253784, 558...",1,[],-1,"[-0.011824905, -0.043365464, 0.008865234, -0.0..."
2,attention-is-all-you-need.pdf,15,4,193,82b50c62-ad6d-4c6e-94f5-128d5c8141dc,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:36:45.815652,53.907655,attention-is-all-you-need.pdf,Attention Is All You Need\nNiki Parmar ∗ Googl...,$.main-text[5],1,"[323.45776367, 524.57171631, 407.69692993, 558...",2,[],-1,"[-0.024725974, -0.052615978, -0.017455006, -0...."


## Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>




In [4]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_1_dpk.db


# Create A Collection



In [5]:
# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name=MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


milvus_client.create_collection(
    collection_name=MY_CONFIG.COLLECTION_NAME,
    dimension=MY_CONFIG.EMBEDDING_LENGTH,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    auto_id=True
)
print ("✅ Created collection :", MY_CONFIG.COLLECTION_NAME)


✅ Created collection : papers


In [6]:
res = milvus_client.insert(collection_name=MY_CONFIG.COLLECTION_NAME, data=data_df.to_dict('records'))

print('inserted # rows', res['insert_count'])

milvus_client.get_collection_stats(MY_CONFIG.COLLECTION_NAME)

inserted # rows 266


{'row_count': 266}

## Close DB Connection

Close the connection so the lock files are relinquished and other notebooks can access the db

In [7]:
milvus_client.close()

print ("✅ SUCCESS")


✅ SUCCESS


## Test your data by doing a Vector Search

See notebook [vector_search.ipynb](vector_search.ipynb)